In [1]:
import pandas as pd
import numpy as np
import plotly
from plotly import graph_objects as go

from base import DataManager
from pyltv2 import PowerSlope, AutoRegression

In [2]:
mx_data = pd.read_csv('data/mx_6-7.csv')

## Backtesting the new rolling default rate model

In [ ]:
backtest_months=3

In [27]:
m1 = PowerSlope(mx_data, market='mx', ltv_expected='mx_ltv_expected_5-22-22_latest.csv')
m1.forecast = m1.forecast_data(m1.data, min_months=5)
m1.backtest, m1.backtest_report = m1.backtest_data(m1.data, hold_months=backtest_months, min_months=5)

Clean data spans 2020-09 to 2022-02
Total # of cohorts: 18

Backtesting 2 months.
12 cohorts will be backtested.


In [28]:
m2 = AutoRegression(mx_data, market='mx', ltv_expected='mx_ltv_expected_5-22-22_latest.csv')
m2.forecast = m2.forecast_data(m2.data, min_months=1)
m2.backtest, m2.backtest_report = m2.backtest_data(m2.data, hold_months=backtest_months, min_months=1,
                                                  retention_weights=(1, 1.5, 1.5, 2, 2))

Clean data spans 2020-09 to 2022-02
Total # of cohorts: 18

Backtesting 7 months.
11 cohorts will be backtested.


In [29]:
cols = [c for c in m1.backtest_report.columns if 'mape' in c and 'borrower_retention' in c]

m1.backtest_report[cols].mean().sort_values(ascending=False)[:10]

borrower_retention-mape    0.008975
dtype: float64

In [30]:
cols = [c for c in m2.backtest_report.columns if 'mape' in c and 'borrower_retention' in c]

m2.backtest_report[cols].mean().sort_values(ascending=False)[:10]

borrower_retention-mape    0.052145
dtype: float64

In [31]:
m1.plot_cohorts('borrower_retention', 'backtest')

In [35]:
m2.plot_cohorts('default_rate_7dpd', 'backtest')

In [33]:
m1.plot_cohorts('cumulative_ltv_per_original-mape', 'backtest_report')

In [34]:
m2.plot_cohorts('cumulative_ltv_per_original-mape', 'backtest_report')

In [ ]:
cols = [c for c in m1.backtest_report.columns if 'mape' in c]

m1.backtest_report[cols].mean().sort_values(ascending=False)[:10]

In [ ]:
cols = [c for c in m2.backtest_report.columns if 'mape' in c]

m2.backtest_report[cols].mean().sort_values(ascending=False)[:10]

## Backtest by holdout months

In [ ]:
metrics = ['default_rate_7dpd-mape', 'default_rate_365dpd-mape']

m1_scores = []
m2_scores = []

holdout = [1,2,3,4]

#weights = [1, 1]
for t in holdout:

    m1.backtest, m1.backtest_report = m1.backtest_data(m1.data, hold_months=t, min_months=5)
    m2.backtest, m2.backtest_report = m2.backtest_data(m2.data, hold_months=t, min_months=5, 
                                                       weight_actuals=.5, weight_tail=.2)
    
    m1_scores.append(m1.backtest_report[metrics].mean())
    m2_scores.append(m2.backtest_report[metrics].mean())
    
m1_scores = pd.concat(m1_scores, axis=1).T
m2_scores = pd.concat(m2_scores, axis=1).T

m1_scores.index = holdout
m2_scores.index = holdout


In [ ]:
traces = [
    go.Scatter(name='m1_scores', x=m1_scores[metrics[0]].index, y=m1_scores[metrics[0]]),
    go.Scatter(name='m2_scores', x=m2_scores[metrics[0]].index, y=m2_scores[metrics[0]])
]

fig = go.Figure(traces)
fig.update_layout(title=metrics[0])
fig.show()

In [ ]:
traces = [
    go.Scatter(name='m1_scores', x=m1_scores[metrics[1]].index, y=m1_scores[metrics[1]]),
    go.Scatter(name='m2_scores', x=m2_scores[metrics[1]].index, y=m2_scores[metrics[1]])
]

fig = go.Figure(traces)
fig.update_layout(title=metrics[1])
fig.show()

## Rolling 3-mo Backtest

In [ ]:
min_months=2
hold_months=3

# timespan to test over
times = range(6, 13)

for t in times:
    m1 = PowerSlope(mx_data, market='mx')
    m2 = Rolling(mx_data, market='mx', ltv_expected='mx_ltv_expected_5-22-22_latest.csv')
    
    # withhold t months of data
    for c in in m1.data.cohort.unique():
        
        
        
    m1 = PowerSlope(mx_data, market='mx')
    m1.backtest, m1.backtest_report = m1.backtest_data(m1.data, hold_months=hold_months, min_months=min_months)

    m2.backtest, m2.backtest_report = m2.backtest_data(m2.data, hold_months=hold_months, min_months=min_months, 
                                                       weight_actuals=.75, weight_tail=.1)